Recognizing Shapes in Objects with OpenCV
=========================================


Import libraries.

In [1]:
import cv2 # OpenCV lib for cv tasks
import numpy as np # for numerical computing
import os # imports os module to work with file paths & directories

Image filenames.

In [2]:

files = (              # tuple having 4 imgs 
    'subway.jpg',
    'breakfast.jpg',
    'dinner.jpg',
    'building.jpg',
)
f = os.path.join('images', files[0]) # path made by joining "subway.jpg" with dir "images"

Define a function for viewing images.

In [3]:
def view_image(i):
    cv2.imshow('view',i) # to show image 
    #view is the window name 
    #i is the img data to be displayed
    #additional arg: flags: cv2.WINDOW_NORMAL: Allows the user to resize the window (default).
                          # cv2.WINDOW_AUTOSIZE: Automatically adjusts the window size to fit the image.
                          # cv2.WINDOW_FULLSCREEN: Opens the window in fullscreen mode.
                          # cv2.WINDOW_OPENGL: Uses OpenGL backend for rendering.
    cv2.waitKey(0) #wait until a key is pressed
    cv2.destroyAllWindows() # closes all opencv windows

Read an image from file.

In [4]:
i=cv2.imread(f) # reads img from path f  
# if img not read then i=None
view_image(i) #fun call

Inspect image contents

In [5]:
print(i.shape) # (height, width, no of channels R/G/B)
print(i[0,0,:]) # BGR pixel values at top left corner (Range 0-255)

(640, 427, 3)
[22 24  4]


Gray-scale

In [6]:
i_gray=cv2.cvtColor(i,cv2.COLOR_BGR2GRAY) # convert i from BGR to grayscale
print(i_gray.shape) # (height, width) # only 1 channel
print(i_gray[0,0]) # 1 channel => 1 intensity value
view_image(i_gray) # fun call

(640, 427)
18


For edge detection

X gradient (Horizontal direction)

In [8]:
sobelx=cv2.Sobel(i_gray,cv2.CV_64F,1,0) 
# cv2.CV_64F o/p img datatype: for high precision:
    # CV_ => constant defined in OpenCV lib
    # 64F => 64-bit floating-point data type
# 1 0 is for order of derivatives in x & y directions
abs_sobelx=np.absolute(sobelx) # to get gradient magnitude
view_image(abs_sobelx/np.max(abs_sobelx)) # display normalized edge-detected image

Y gradient (Vertical direction)

In [9]:
sobely=cv2.Sobel(i_gray,cv2.CV_64F,0,1)
abs_sobely=np.absolute(sobely)
view_image(abs_sobely/np.max(abs_sobely))

Magnitude of gradient vector (Horizontal & Vertical direction)

In [10]:
magnitude=np.sqrt(sobelx**2 + sobely**2)
view_image(magnitude/np.max(magnitude))

Canny edge detection

In [13]:
edges=cv2.Canny(i_gray,200,250) # low,high threshold values
# Pixels with gradient magnitudes below low threshold are discarded
# pixels with gradient magnitudes above high threshold are considered strong edges
view_image(edges)

Theory: Hough transforms

![title](images2/line_diagram.png)

![title](images2/accumulator1.png)

![title](images2/edge_pixel.png)

![title](images2/accumulator2.png)

![title](images2/accumulator3.png)


Hough transform for lines

In [14]:
lines=cv2.HoughLinesP(
edges,
rho=1, # Distance resolution of accumulator in pixels # smaller value higher accuracy but higher computational complexity # default 1 pixel
theta=1. * np.pi/100.0, #resolution of angle measurement in radians # smaller value higher accuracy but higher computational complexity # default pi/180
threshold=20, # minimum number of votes (intersections in Hough space) required to detect a line
# lower value more lines but can be noisy
# higher value less no of lines
minLineLength=25, # min len for valid line (reduce noise)
maxLineGap=5,) # max gap btw segments to be treated as single line # gap less than this would merge segments into single line
# higher value => more segments connected => no of lines detected less
i_lines=i.copy() # copy of original img
for l in lines:
    x1,y1,x2,y2=l[0]
    cv2.line(i_lines,(x1,y1),(x2,y2),(0,0,255),thickness=3) # img data, 2 coordinates for line, line color, line thickness
view_image(i_lines)

Hough transform for circles

In [15]:
# gives irrelevant circles as well...do on blur img
circles=cv2.HoughCircles(
i_gray,
method=cv2.HOUGH_GRADIENT,
dp=2, #  ratio of img resolution to accumulator resolution 
# 2 => accumulator has half resolution of i/p img (speed up computation)
minDist=35, # min distance btw centers of detected circles (avoid detecting same circle again) 
# if distance btw 2 circle centres is less than this then weaker is discarded
param1=150, # higher threshold of Canny edge detector used in circle detection algo
# controls sensitivity of edge detection (higher value =>fewer circles detected)
param2=40, # represents accumulator threshold for circle detection
# controls min no of votes reqd for circle to be detected (higher value =>fewer circles detected)
minRadius=15,
maxRadius=25)
i_circles=i.copy()
for x,y,r in circles[0]:
    cv2.circle(
    i_circles,
    (x,y), # center coordinates
    int(r), # radius
    (0,0,255), # color of circle
    thickness=3) # circle outline
    view_image(i_circles)

Blur the image first

In [16]:
i_blurred=cv2.GaussianBlur(
i_gray,
ksize=(21,21), #  kernel size 21x21 pixels # how much smoothing is applied to img # Larger kernel sizes => more smoothing.
sigmaX=0,) # sd of Gaussian distribution in x direction
# 0 => OpenCV should automatically calculate sd based on kernel size (21, 21)
# sd controls extent of blurring......larger values =>more blurring
view_image(i_blurred)

Circle detection on blurred image

In [17]:
circles=cv2.HoughCircles(
i_blurred,
method=cv2.HOUGH_GRADIENT,
dp=2,
minDist=35,
param1=150,
param2=40,
minRadius=15,
maxRadius=25)
i_circles=i.copy()
for x,y,r in circles[0]:
    cv2.circle(
    i_circles,
    (x,y),
    int(r),
    (0,0,255),
    thickness=3)
    view_image(i_circles)